<a href="https://colab.research.google.com/github/aloml2543/KB_OCR/blob/main/src/model5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %cp /content/drive/MyDrive/KB_OCR/DATA/data.yaml /content/
%cp /content/drive/MyDrive/KB_OCR/DATA/mk_img.zip /content/

In [ ]:


"""
출처
https://codesample-factory.tistory.com/1343 > 이미지 노이즈 처리
https://www.youtube.com/watch?v=T0DO1C8uYP8 > 빵형의 개발도상국
https://www.mcst.go.kr/kor/s_policy/subPolicy/contents/contents09.jsp > 안심 글꼴

이미지 검색
> https://unsplash.com/s/photos/pixel
> https://pixabay.com/

이미지 출처
><a href="https://pixabay.com/ko/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=2658014">Pixabay</a>로부터 입수된 <a href="https://pixabay.com/ko/users/davidzydd-985081/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=2658014">David Zydd</a>님의 이미지 입니다.
"""
!pip install jamo
!pip install hangul_utils
!pip install parmap
!pip install wget
import warnings
import wget
import os
import cv2
import random
import parmap
import yaml
import multiprocessing
import numpy as np
from tqdm import tqdm
from glob import glob
from jamo import j2hcj, h2j
from sklearn.cluster import KMeans 
from PIL import ImageFont, ImageDraw, Image
from google.colab.patches import cv2_imshow
from hangul_utils import split_syllables,join_jamos
from scipy.cluster.hierarchy import dendrogram, linkage
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
dir_ = os.getcwd() + '/dataset/Font/'
URL = "https://onedrive.live.com/download?cid=44F5C59EE841EE0A&resid=44F5C59EE841EE0A%21244402&authkey=ABfBdDi5elNBoss"

if not(os.path.isdir(dir_)):
  os.makedirs(os.path.join(dir_))

file_ = os.getcwd() + '/dataset/Font.zip'
wget.download(URL, file_)

dir_ = os.getcwd() + '/dataset/Font'
!unzip -n '{file_}'  -d '{dir_}'

try:
  os.remove(dir_ + '/- (146).ttf')
  os.remove(dir_ + '/- (148).ttf')
  os.remove(dir_ + '/- (149).ttf')
except:
  pass

file_ = os.getcwd() + '/dataset/data.yaml'
URL="https://onedrive.live.com/download?cid=44F5C59EE841EE0A&resid=44F5C59EE841EE0A%21244403&authkey=ACcKIYIEAbe10lY"
wget.download(URL, file_)

In [ ]:
class hangle_img():
  def __init__(self):
    self.img_loc = os.getcwd() + '/dataset/images/'
    self.label_loc = os.getcwd() + '/dataset/labels/'
    self.font_dir = os.getcwd() + '/dataset/Font/'
    
    if not(os.path.isdir(self.img_loc)):
      os.makedirs(os.path.join(self.img_loc))
    if not(os.path.isdir(self.label_loc)):
      os.makedirs(os.path.join(self.label_loc))
    
    self.font_list = os.listdir(self.font_dir)
    if '.ipynb_checkpoints' in self.font_list:
      font_list.remove('.ipynb_checkpoints')

    self.info = {
        'char_dic': {'ㄱ': 144,'ㄲ': 145,'ㄴ': 146,'ㄷ': 147,'ㄹ': 148,'ㅁ': 149,'ㅂ': 150,'ㅅ': 151,'ㅆ': 152,'ㅇ': 153,'ㅈ': 154,'ㅊ': 155,'ㅋ': 156,'ㅌ': 157,'ㅍ': 158,'ㅎ': 159},
        '0' : ''' ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,/?'";:][}{)(<>+=-_*&^%$#@!~\\ㄳㄵㄶㄺㄻㄼㄽㄾㄿㅀㅄ''',
        'w_0' : [10,2,10,2,4,6,10,10,2,10,2,10,10,2,10,4,4,4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        '1' : """ ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ""",
        'w_1' : [5,5,3,3,2,5,3,2,1,5,3,2,2,3,5,3,1,1,3,4,1,3,],
        '2' : """ ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ""",
        'w_2' : [10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        'only_0' : """ㄳㄵㄶㄺㄻㄼㄽㄾㄿㅀㅄㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,/?'";:][}{)(<>+=-_*&^%$#@!~\\""",
        'not_kor' : ''' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,/?'";:][}{)(<>+=-_*&^%$#@!~\\''',
        'ans_cha' : ''' ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,/?'";:][}{)(<>+=-_*&^%$#@!~\\ㄳㄵㄶㄺㄻㄼㄽㄾㄿㅀㅄ''',
        'char_dic' : {144: 'ㄱ',145: 'ㄲ',146: 'ㄴ',147: 'ㄷ',148: 'ㄹ',149: 'ㅁ',150: 'ㅂ',151: 'ㅅ',152: 'ㅆ',153: 'ㅇ',154: 'ㅈ',155: 'ㅊ',156: 'ㅋ',157: 'ㅌ',158: 'ㅍ',159: 'ㅎ'},
        'name' : ['space','giuk','Xgiuk','nien','diged','Xdiged','lier', 'miem', 'biep', 'Xbiep', 'siot', 'Xsiot', 'ieng', 'jied', 'Xjied', 'chied', 'kieck', 'tiet', 'piep', 'hied', 'AH', 'AEH-I', 'YAH', 'YAEH-I', 'EOH', 'AEH-O', 'YEOH', 'YAEH-O', 'OH', 'OH-AH' ,'OH-AEH-I', 'OH-EE', 'YOH', 'OO', 'OO-EOH', 'OO-AEH-O', 'OO-EE', 'YOO', 'EH', 'EH-EE' ,'EE', 'a' ,'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','.', ',', '/', '?', "'",'"', ';',':',']','[','}','{',')','(','<', '>', '+', '=', '-', '_', '*', '&', '^', '%', '$', '#', '@', '!', '~', '\\', '_gisi', '_niji', '_nihi', '_ligi', '_limi', '_libi', '_lisi', '_liti' ,'_lipi', '_lihi', '_bisi', '_giuk', '_Xgiuk', '_nien', '_diged', '_lier', '_miem', '_biep', '_siot', '_Xsiot', '_ieng', '_jied', '_chied', '_kieck', '_tiet', '_piep', '_hied', 'letter']
    }
    

  def ri(self, x, y):
    return random.randint(x,y)

  def _get_data(self, data=None):
    word = ''
    if data == None:
      length = self.ri(1, 20)
      for n in range(length):
        letter_ = ''
        for jong in range(3):
          letter_ += random.choices(self.info[str(jong)], weights=self.info["w_"+str(jong)])[0]
          if letter_[jong] == " " :
            break
          if jong == 0:
            if letter_[jong] in self.info['only_0']:
              break
        word += join_jamos(letter_)
    else:
      length = len(data)
      word = data
    
    while True:
      if word == '':
        break
      if word[0] == ' ':
        word = word[1:]
      else:
        break
    length = len(word)
    while True:
      if word == '':
        break
      if word[length-1] == ' ':
        word = word[:length-1]
        length = len(word)
      else:
        break

    for i in reversed(range(1, len(word)-2)):
      
      if word[i] == ' ':
        if word[i-1] == ' ':
          word = word[:i]+word[i+1:]
          length = len(word)
    
    if word == '':
      raise Exception('\nword를 만들던 중 길이가 0인 데이터가 발생했습니다. 다음인덱스로 넘어갑니다.')
  
    letter_size = 50
    
    word_width = length * letter_size + 30 #margin for font's irregular position
    back_img_size = (word_width+self.ri(1, 20),word_width+self.ri(1, 20),3) #height, width, channel

    r_font = random.choices(self.font_list)[0]
    font = ImageFont.truetype(self.font_dir + r_font, size=letter_size)
    w_li_ = []
    for word_ in ['가', '가 가','가가']:

      img_ = np.zeros(back_img_size, dtype=np.uint8)
      img_ = Image.fromarray(img_)
      draw = ImageDraw.Draw(img_)
      draw.text((5,5), word_, font=font, fill=(256,256,256))
      img_ = np.array(img_)
      l, r, _, _ = self._find_pos(img_)
      w_li_.append(r - l)
    font_term = 5 - l
    word_term = w_li_[2] - w_li_[0] - w_li_[0]
    space_term = w_li_[1] - w_li_[0] - w_li_[0] + 2

    color_1 = (self.ri(0,255), self.ri(0,255), self.ri(0,255))
    color_2 = (self.ri(0,255), self.ri(0,255), self.ri(0,255))
    while color_1 == color_2:
      color_2 = (self.ri(0,255), self.ri(0,255), self.ri(0,255))

    img_style = (color_1,color_2)

    word_rotation = self.ri(-90, 90)

    r_data = {
        'length': length,
        'font': font,
        'font_name': r_font,
        'font_term': font_term,
        'word_term': word_term,
        'space_term': space_term,
        'letter_size': letter_size,
        'word': word,
        'word_rotation': word_rotation,
        'back_img_size': back_img_size,
        'img_style': img_style,
        'rect_pos': []
    }
    return r_data

  def _save(self, num, r_data):
    with open(self.label_loc + str(num) + '.txt', mode='wt', encoding='utf-8') as f:
      h, w, c = r_data['img'].shape
      r_data['rect_pos'] = np.array(r_data['rect_pos'], dtype=np.int64)
      for i, letter in enumerate(r_data['word']):
        rect = r_data['rect_pos'][i]
        x = round(((rect[1] - rect[0])/2 + rect[0])/w, 6)
        y = round(((rect[3] - rect[2])/2 + rect[2])/h, 6)
        height = round((rect[3] - rect[2])/h, 6)
        width = round((rect[1] - rect[0])/w, 6)
        split_letter = split_syllables(letter)
        len_ = len(split_letter) 
        for pos_ in reversed(range(len_)):
          if pos_ == 2:
            if split_letter[pos_] in self.info['char_dic']:
              index = self.info['char_dic'][split_letter[pos_]]
            else:
              try:
                index = self.info['0'].index(split_letter[pos_])
              except:
                raise Exception(num, "번쨰 데이터에서 charecter에 없는 데이터('",s_thing,"')가 발생했습니다. 추가해주세요.")
            f.write("{} {:.06f} {:.06f} {:.06f} {:.06f}\n".format(index, x, y+0.000001, width, height))

          elif pos_ == 0:
            try:
              index = self.info['0'].index(split_letter[pos_])
            except:
              raise Exception(num, "번쨰 데이터에서 charecter에 없는 데이터('",s_thing,"')가 발생했습니다. 추가해주세요.")
            f.write("{} {:.06f} {:.06f} {:.06f} {:.06f}\n".format(index, x-0.000001, y, width, height))
          elif pos_ == 1:
            try:
              index = self.info['0'].index(split_letter[pos_])
            except:
              raise Exception(num, "번쨰 데이터에서 charecter에 없는 데이터('",s_thing,"')가 발생했습니다. 추가해주세요.")
            f.write("{} {:.06f} {:.06f} {:.06f} {:.06f}\n".format(index, x, y, width+0.000001, height))

        f.write("{} {:.06f} {:.06f} {:.06f} {:.06f}\n".format(160, x, y, width, height))
    cv2.imwrite(self.img_loc + str(num)+'.jpg', r_data['img'])

  def _error(self, error_code, data=None):
    if error_code == 'InputError':
      raise Exception("InputError\n\n가능한 버전이 이렇게 있습니다\n1. 인덱스(list-int)와 data(list-str) 특정\n2. data(list-str)만 특정 (index=None)\n3. index(int)개수 만큼의 랜덤 데이터(data=None)\n\n위의 형식에 맞게 데이터를 넣어주세요\n>> start(self, index, data = None, show_img=False)")
    
    if error_code == 'CropError':
      raise Exception('\n\nCropError.문자 위치 특정 작업을 실패했습니다. \nlength:',data['length'],'\nword:',data['word'],'\n\n다음 인덱스로 넘어갑니다.')

  def _find_pos(self, img, r_data=None):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(img_gray,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_xy = np.array(contours)
    x_value = []
    y_value = []
    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            x_value.append(contours_xy[i][j][0][0]) #네번째 괄호가 0일때 x의 값
            y_value.append(contours_xy[i][j][0][1]) #네번째 괄호가 0일때 x의 값
    try:
      x_min = min(x_value)
      x_max = max(x_value)
      y_min = min(y_value)
      y_max = max(y_value)
    except ValueError:
      raise Exception()

    if r_data == None:
      return x_min, x_max, y_min, y_max # left, right, up, down
    else:
      r_data['rect_pos'].append([x_min, x_max, y_min, y_max])



  def _draw_rect(self, img, pt1, pt2, color = (255,255,255)):
    cv2.rectangle(img, pt1, pt2, color, 1)
    return img

  def _draw_text(self, img, text, point, r_data):
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    draw.text(point, text, font=r_data['font'], fill=(256,256,256))
    img = np.array(img)
    return img

  def _make_img(self, r_data):
    img = np.zeros(r_data['back_img_size'], dtype=np.uint8)
    start_y = int(r_data['back_img_size'][0]/2 - r_data['letter_size']/2)
    start_x = r_data['letter_size']
    r_data['rect_pos'] = []
    term_ = 0
    for letter in r_data['word']:
      if letter == ' ':
        img_ = np.zeros(r_data['back_img_size'], dtype=np.uint8)
        img_ = self._draw_rect(img_, (start_x+term_, start_y), (start_x+term_+r_data['space_term'], start_y+r_data['letter_size']+2))
        try:
          left, right, up, down = self._find_pos(img_)
        except:
          self._error("CropError", data=r_data)
        term_ = int(term_ + right - r_data['font_term'] - left)
        img_ = self._give_style(r_data, img_)
        self._find_pos(img_, r_data=r_data)

      else:
        img_ = np.zeros(r_data['back_img_size'], dtype=np.uint8)
        img_ = self._draw_text(img_, letter, (start_x+term_, start_y), r_data)
        try:
          left, right, up, down = self._find_pos(img_)
        except:
          self._error("CropError", data=r_data)
        if left == None:
          raise Exception('현재 letter(', letter, ')가 표현이 되지 않습니다.\n폰트:',r_data['font_name'],' \n다음 인덱스로 넘어갑니다')
        term_ = int(term_ + right - r_data['font_term'] + r_data['word_term'] - left)
        img_ = self._give_style(r_data, img_)
        self._find_pos(img_, r_data=r_data)
        img = cv2.bitwise_or(img, img_)

    r_data['word_pos'] = self._find_pos(img)
    r_data['img'] = img
  
  def _crop_img(self, r_data, margin = 20):
    left, right, up, down = r_data['word_pos']
    if up - margin < 0 or left - margin < 0:
      return
    r_data['img'] = r_data['img'][up - margin : down + margin, left - margin: right + margin, :].copy()
    r_data['rect_pos'] = np.array(r_data['rect_pos'], dtype=np.float64)
    sum_ = np.array([-left + margin, -left + margin, -up + margin, -up + margin])
    r_data['rect_pos'] += sum_

  def _draw_pos(self, r_data):
    img = r_data['img'].copy()
    r_data['rect_pos'] = np.array(r_data['rect_pos'], dtype=np.int64)
    for rect in r_data['rect_pos']:
      img = self._draw_rect(img, (rect[0], rect[2]), (rect[1], rect[3]), color=(255,0,0))
    print(r_data['word'])
    return img
  
  def _resize(self, r_data, size = 200):
    height, width, channel = r_data['img'].shape
    r_data['rect_pos'] = np.array(r_data['rect_pos'], dtype=np.float64)
    if height <= size:
      if width <= size:
        return
    if height > width:
      ratio_ = size / height
      size_ = (int(ratio_*width),size)
    else:
      ratio_ = size / width
      size_ = (size, int(ratio_*height))
    r_data['img'] = cv2.resize(r_data['img'], size_)
    r_data['rect_pos'] *= ratio_
  
  def _over_img(self, r_data):
    height, width, channel = r_data['back_img_size']
    back = np.full((height, width, channel), r_data['img_style'][0])
    text = np.full((height, width, channel), r_data['img_style'][1])
    text = (text / 250) * r_data['img']
    r_data['img'] = cv2.add(back,text,dtype=cv2.CV_64F)

    for _ in range(self.ri(0,(height*width)//20000)):
      pt1 = (self.ri(0, height), self.ri(0, height))
      pt2 = (self.ri(0, width), self.ri(0, width))
      color = (self.ri(0,255), self.ri(0,255), self.ri(0,255))
      r_data['img'] = cv2.line(r_data['img'], pt1, pt2, color, thickness=self.ri(1,2))
    for _ in range(self.ri(0,(height*width)//2000)):
      center = (self.ri(0, width), self.ri(0, height))
      color = (self.ri(0,255), self.ri(0,255), self.ri(0,255))
      r_data['img'] = cv2.circle(r_data['img'], center, self.ri(0, 5), color, thickness=self.ri(1,2))

  def start(self, index, data = None, show=False):
    if type(index)==int:
        index = range(index)
    elif type(index)==str:
      self._error('InputError')
        
    elif type(index) == list or type(index) == range:
      if type(index[0]) == str:
        self._error('InputError')
      elif data == None:
        pass
      else:
        if len(index) != len(data):
          raise Exception("데이터와 인덱스의 길이가 다릅니다.")
  
    for i in tqdm(index):
      try:
        if data == None:
          r_data = self._get_data(None)
        else:
          r_data = self._get_data(str(data[index.index(i)]))

        self._make_img(r_data)
        self._over_img(r_data)
        self._crop_img(r_data)
        self._resize(r_data)
        self._save(i, r_data)

        #cv2_imshow(self._draw_pos(r_data))

      except Exception as e:
        print('\n', str(e))

  def make_word(self, data, length):
    data = np.array(data)
    try:
      kmeans = KMeans(n_clusters=length)
      kmeans.fit(data[:,0:1])
      pred = kmeans.predict(data[:,0:1])
    except:
      return False

    min_ = min(pred)
    max_ = max(pred)
    word=[]

    for i in range(min_, max_+1):
      pos = [j for j in range(len(pred)) if pred[j]==i]
      temp = []
      for j in pos:
        temp.append(list(data[j,:]))
      word.append(temp)

    ans = []
    word.sort(key=lambda x:x[0])
    for letter in word:
      letter.sort(key=lambda x:x[1])
      temp = [int(i[2]) for i in letter]
      if len(temp) == 1:
        temp = self.info['ans_cha'][temp[0]]
      elif len(temp) == 2:
        temp = [self.info['ans_cha'][i] for i in temp]
        if temp[0] in 'ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ':
          temp[0], temp[1] = temp[1], temp[0]
      elif len(temp) == 3:
        temp_ = []
        for i in range(len(temp)):
          if temp[i] < 133:
            temp_.append(self.info['ans_cha'][temp[i]])
          elif temp[i] < 144:
            jong = self.info['ans_cha'][temp[i]]
          else:
            jong = self.info['char_dic'][temp[i]]
          
        if temp_[0] in 'ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ':
          temp_[0], temp_[1] = temp_[1], temp_[0]
        if jong:
          temp_.append(jong)
        temp = temp_
      ans.append(join_jamos("".join(temp)))
      
    return "".join(ans)
    
generater = hangle_img()

In [ ]:
# num_cores = multiprocessing.cpu_count()
# index_list = range(30000)
# splited_index = np.array_split(index_list, num_cores)

# parmap.map(generater.start ,splited_index,pm_pbar=True,pm_processes=num_cores)
# print('Done!!')

# save_path = os.getcwd()+'/mk_img.zip'
# %cd dataset
# !zip '{save_path}' -r ./*
# %cd ..

save_path = os.getcwd()+'/mk_img.zip'
to_ = os.getcwd()+'/dataset'
!unzip -n '{save_path}' -d '{to_}'

In [ ]:
img_list = glob(os.getcwd()+"/dataset/images/*.jpg")
print(len(img_list))
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state=2000)
print(len(train_img_list), len(val_img_list))

with open(os.getcwd()+"/dataset/train.txt", "w") as f:
  f.write('\n'.join(train_img_list)+'\n')
with open(os.getcwd()+"/dataset/val.txt", "w") as f:
  f.write('\n'.join(val_img_list)+'\n')

with open(os.getcwd() + '/dataset/data.yaml', 'r') as f:
  data = yaml.load(f)

data['train'] = os.getcwd() + "/dataset/train.txt"
data['val'] = os.getcwd() + "/dataset/val.txt"
data['nc'] = 161
data['names'] = ['space','giuk','Xgiuk','nien','diged','Xdiged','lier', 'miem', 'biep', 'Xbiep', 'siot', 'Xsiot', 'ieng', 'jied', 'Xjied', 'chied', 'kieck', 'tiet', 'piep', 'hied', 'AH', 'AEH-I', 'YAH', 'YAEH-I', 'EOH', 'AEH-O', 'YEOH', 'YAEH-O', 'OH', 'OH-AH' ,'OH-AEH-I', 'OH-EE', 'YOH', 'OO', 'OO-EOH', 'OO-AEH-O', 'OO-EE', 'YOO', 'EH', 'EH-EE' ,'EE', 'a' ,'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','.', ',', '/', '?', "'",'"', ';',':',']','[','}','{',')','(','<', '>', '+', '=', '-', '_', '*', '&', '^', '%', '$', '#', '@', '!', '~', '\\', '_gisi', '_niji', '_nihi', '_ligi', '_limi', '_libi', '_lisi', '_liti' ,'_lipi', '_lihi', '_bisi', '_giuk', '_Xgiuk', '_nien', '_diged', '_lier', '_miem', '_biep', '_siot', '_Xsiot', '_ieng', '_jied', '_chied', '_kieck', '_tiet', '_piep', '_hied', 'letter']                                    

with open(os.getcwd() + '/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd /
%cd /content/yolov5
!pip install -r requirements.txt

# %load_ext tensorboard
# %tensorboard --logdir /content/yolov5/runs/
!nvidia-smi

# %cp /content/drive/MyDrive/KB_OCR/model5_4/last.pt /content/yolov5

In [ ]:
%cp /content/drive/MyDrive/KB_OCR/model5_4/* /content/yolov5

In [ ]:
!python train.py --img 224 --batch 100 --epoch 1000 --data /content/dataset/data.yaml --cfg ./models/yolov5x.yaml --weights last.pt --name OCR_result

In [ ]:
33

In [ ]:
%cp /content/yolov5/runs/train/OCR_result2/weights/* /content/drive/MyDrive/KB_OCR/model5_4

In [ ]:
%cd /content
val_img_path= os.getcwd() + '/dataset/images/'
weights_path = os.getcwd() + '/yolov5/best.pt'
%cd yolov5/
!python detect.py --weights '{weights_path}' --img 224 --conf 0.3 --source "{val_img_path}" --save-txt --line-thicknes 1
%cd ..

label_list = glob(os.getcwd() + '/yolov5/runs/detect/exp2/labels/*.txt')

for lable_dir in label_list:
  with open(lable_dir, 'r') as f:
    lines = f.readlines()
  data, length, num = [], 0, lable_dir.split('/')[-1]
  for line in lines:
    l_list =line.split(" ")
    if int(l_list[0]) == 160:
      length += 1
      continue
    data.append([float(l_list[1]), float(l_list[2]), int(l_list[0])])
  ans = generater.make_word(data, length)
  print(num, ' : ',ans)

In [ ]:
label_list = glob(os.getcwd() + '/yolov5/runs/detect/exp2/labels/*.txt')

for lable_dir in label_list:
  with open(lable_dir, 'r') as f:
    lines = f.readlines()
  data, length, num = [], 0, lable_dir.split('/')[-1]
  for line in lines:
    l_list =line.split(" ")
    if int(l_list[0]) == 160:
      length += 1
      continue
    data.append([float(l_list[1]), float(l_list[2]), int(l_list[0])])
  ans = generater.make_word(data, length)
  print(num, ' : ',ans)

In [ ]:
img = cv2.imread("/content/dataset/images/", cv2.IMREAD_COLOR)